In [2]:
import uuid
import random
import pandas as pd
import numpy as np
import datetime
from datetime import timedelta

#  path modification to locate helpers module
import sys
import os
project_path = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_path not in sys.path:
    sys.path.append(project_path)

from helpers import create_table_leanx as ctl, helpers
import values.values_default as values
from content_generators import master_data, text_data, sales_doc_data

In [13]:
# text tables
for method in (
    text_data.domain_fixed_values, 
    text_data.sales_doc_types, 
    text_data.sales_organizations,
    text_data.distribution,
    text_data.sales_doc_item_categories,
    text_data.sales_doc_rejection_reasons,
    text_data.system_status,
    text_data.blocking_reasons,
    text_data.releases,
    text_data.purchasing_doc_types,
    text_data.purchasing_organizations
):
    table_dict = method()
    for k, v in table_dict.items():
        table_name = k.split('_')[0]
        all_cols = pd.DataFrame(columns=[c[0] for c in ctl.fetch_table(table_name)])
        df = pd.concat([all_cols, pd.DataFrame(v.values())])

        directory = os.path.dirname(f'../data/p2p/text/{table_name}.csv')
        if not os.path.exists(directory):
            os.makedirs(directory)

        df.to_csv(f'../data/p2p/text/{table_name}.csv', index=False)

In [14]:
# master tables
for method in (
    master_data.users,
    master_data.customers_and_vendors, 
    master_data.plants, 
    master_data.materials, 
    master_data.material_support,
    master_data.routes,
    master_data.company_codes
):
    table_dict = method()
    for k, v in table_dict.items():
        table_name = k.split('_')[0]
        all_cols = pd.DataFrame(columns=[c[0] for c in ctl.fetch_table(table_name)])
        df = pd.concat([all_cols, pd.DataFrame(v.values())])

        directory = os.path.dirname(f'../data/p2p/master/{table_name}.csv')
        if not os.path.exists(directory):
            os.makedirs(directory)

        df.to_csv(f'../data/p2p/master/{table_name}.csv', index=False)

/var/folders/1q/whzv5lqd6sjb2d4mnmydzgj40000gn/T/ipykernel_51454/2379721070.py:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([all_cols, pd.DataFrame(v.values())])
/var/folders/1q/whzv5lqd6sjb2d4mnmydzgj40000gn/T/ipykernel_51454/2379721070.py:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([all_cols, pd.DataFrame(v.values())])


In [8]:
def get_user_name(automation_probability: float):
    if random.random() <= automation_probability:
        return 'BATCH_JOB'
    return random.choice(list(values.om_users.keys()))

def get_time_consumption(start_date, planned_target_date, latest_date):
    return (latest_date - start_date) / (planned_target_date - start_date)

In [15]:
MARC = pd.read_csv('../data/p2p/master/MARC.csv')
MARA = pd.read_csv('../data/p2p/master/MARA.csv')
MAKT = pd.read_csv('../data/p2p/master/MAKT.csv')
LFB1 = pd.read_csv('../data/p2p/master/LFB1.csv')
LFA1 = pd.read_csv('../data/p2p/master/LFA1.csv')

all_prices = {}
all_has_contract_probabilities = {}
all_matnr_names = {}
all_availabilities = {}

# get price and material group by MATNR
for nr in MARA['MATNR']:
    name = MAKT[MAKT['MATNR'] == nr]['MAKTX'].values[0]
    for k, v in values.om_material_groups.items():
        for mgrp, attr in v.items():
            for mat, details in attr['materials'].items():
                if name == mat:
                    all_prices[nr] = details['price']
                    all_matnr_names[nr] = mat
                    all_has_contract_probabilities[nr] = attr['has_contract_probability']
                    all_availabilities[nr] = attr['availability']

In [18]:
def get_params():
    # basic informaiton
    company_code = random.choice(list(values.om_company_codes.keys()))
    plant = random.choice(values.om_company_codes[company_code]['plants'])
    konnr = str(uuid.uuid4())[-15:]
    lifnr = random.choice(list(LFB1[LFB1['BUKRS'] == company_code]['LIFNR']))
    vendor_name = LFA1[LFA1['LIFNR'] == lifnr]['NAME1'].values[0]
    purchasing_org=random.choice(values.om_plants[plant]['purchasing_orgs'])
    new_lifnr = random.choice(list(LFB1[LFB1['BUKRS'] == company_code]['LIFNR']))
    payment_term = random.choice(list(LFB1[LFB1['LIFNR'] == lifnr]['ZTERM']))
    requested_by = get_user_name(0.1)
    has_creditmemo = True if random.random() > 0.8 else False

    # materials related
    all_matnrs = MARC[MARC['WERKS'] == plant]['MATNR'].unique()
    matnrs = random.sample(list(all_matnrs), min(random.randint(5, 25), len(all_matnrs)))

    # quantities Sourcream
    quantity_factor = random.randint(12, 24) if values.om_plants[plant]['high_value'] else random.randint(3, 9)
    quantities = [random.randint(6, 15)*quantity_factor for temp_quantity_index in range(len(matnrs))]

    # quantities Meritor
    # quantity_factor = random.randint(3, 5) if values.om_plants[plant]['high_value'] else random.randint(1, 3)
    # quantities = [random.randint(3, 6)*quantity_factor for temp_quantity_index in range(len(matnrs))]
    has_qty_change = True if values.om_company_codes[company_code]['incorrect_qty_prbobability'] > 0.3 else False

    # prices and contract
    prices = [all_prices[matnrs[temp_matnr_index]] for temp_matnr_index in range(len(matnrs))]
    has_contract_probabilities = [all_has_contract_probabilities[matnrs[temp_contract_index]] for temp_contract_index in range(len(matnrs))]
        #contract prob based on materials and purchasing org
    item_has_contract = [False if ((random.random() > has_contract_probabilities[j]) or (random.random() > 0.57 and purchasing_org == 'JAPC'))  else True for j in range(len(has_contract_probabilities))]
    has_contract = True if random.random() > 0.09 else False
    priceifnocontract = [round(all_prices[matnrs[temp_matnr_index]] * (random.uniform(1.01, 1.039)),4) for temp_matnr_index in range(len(matnrs))]
    # for hc in item_has_contract:
    #     if hc == False:
    #         has_contract = False
    #         break
    
    # delivery status
    delivery_status = [] # early: -x | on_time: 0 | late: x
    for temp_delivery_status_index in range(len(matnrs)):
        extra_days = 0
        if lifnr in ('VND1','VND6','VND14') and random.random() > 0.1:
             extra_days = random.randint(5, 17) # late
        elif all_availabilities[matnrs[temp_delivery_status_index]] < 0.4: 
            extra_days = random.randint(2, 14) # late
        elif (all_availabilities[matnrs[temp_delivery_status_index]] >= 0.4) and (all_availabilities[matnrs[temp_delivery_status_index]] < 0.7):
            extra_days = 0 # on time
        else:
            extra_days = -random.randint(3, 6)
        delivery_status.append(extra_days)
        
    # free text
    is_free_text = True if random.random() < values.om_company_codes[company_code]['free_text_pr_probability'] else False
    free_text_materials = [all_matnr_names[matnr] for matnr in matnrs] # TODO change with material description

    # time distribution
    pr_req_years = [2023, 2024]
    pr_req_year_probability = np.array([0.5, 0.5])
    pr_req_year_probability = pr_req_year_probability / pr_req_year_probability.sum(axis=0, keepdims=True)
    pr_req_year = pr_req_years[np.random.choice(2, p=pr_req_year_probability)]
    if pr_req_year == 2023:
        pr_req_month_distribution = np.array([7, 5, 5, 4, 2, 6, 3, 5, 5, 6, 7, 10])
        pr_req_month_probability = pr_req_month_distribution / pr_req_month_distribution.sum(axis=0, keepdims=True)
        pr_req_month = np.random.choice(12, p=pr_req_month_probability) + 1 # because months should be in [1, 12] for datetime
    else:
        pr_req_month_distribution = np.array([7, 5, 5, 4, 2, 6, 3, 5, 7, 8, 9, 9])
        pr_req_month_probability = pr_req_month_distribution / pr_req_month_distribution.sum(axis=0, keepdims=True)
        pr_req_month = np.random.choice(12, p=pr_req_month_probability) + 1 # because months should be in [1, 12] for datetime

    #cashdiscount and payment days

    cashdiscount = random.uniform(0.50, 1.50)
    paymentdays = random.randint(15, 45)
    
    #pr or no pr  / po or no po

    has_pr = True if random.random() > 0.15 else False
    has_po = True if random.random() > 0.2 else False
    # automationprobs for events 
    autoprobs = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
    if plant in ('PL10', 'PL07','PL05'):
        autoprobs = [x / 2 for x in autoprobs]

    params = {
        'matnrs': matnrs,
        'konnr': konnr,
        'lifnr': lifnr,
        'plant': plant,
        'quantities': quantities,
        'prices': prices, # considered contract and PO prices
        'priceifnocontract' : priceifnocontract,
        'company_code': company_code,
        'purchasing_org': purchasing_org,
        'payment_term': payment_term,
        'requested_by': requested_by,
        'has_creditmemo': has_creditmemo,
        
        'item_has_contract': item_has_contract,
        'has_contract': has_contract,
        'is_free_text': is_free_text,
        'free_text_materials': free_text_materials,

        'has_qty_change': has_qty_change,

        'has_payment_term_change': random.random() > values.proc_vendors[vendor_name]['payment_term_stsability'],
        'delivery_status': delivery_status,

        'new_payment_term': 'Z090',
        'new_vendor': new_lifnr,
        'has_pr' : has_pr,
        'has_po' : has_po,
        'pr_req_year': pr_req_year,
        'pr_req_month': pr_req_month,
        'autoprobs' : autoprobs,
        'cashdiscount' : cashdiscount,
        'paymentdays' : paymentdays 
    }

    return params

In [ ]:
purchasing_doc_tables = {
	'BKPF_json': {},
	'BSEG_json': {},
	'EBAN_json': {}, 
	'CDHDR_json': {},
	'CDPOS_json': {},
	'EKES_json' : {},
	'EKKO_json': {},
	'EKPO_json': {},
	'NAST_json': {},
	'MSEG_json': {},
	'EKBE_json': {},
	'RBKP_json': {},
	'RSEG_json': {},
	'EKET_json': {},
}

for i in range(37527):
# for i in range(1_000):
	transition_prob = np.array([
	  # [0     1     2     3     4     5     6     7     8     9     10    11    12    13    14    15    16    17    18  ] 
		[0.00, 0.50, 1.30, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.05], # 0 Create Contract, PR, PO
		[0.00, 0.00, 1.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.10], # 1 Approve PO
		[0.00, 0.00, 0.00, 1.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.10], # 2 Send PO
		[0.00, 0.00, 0.00, 0.00, 1.00, 0.00, 0.00, 0.00, 0.00, 0.01], # 3 Post Goods Receipt
		[0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 1.00, 0.15], # 4 Create Vendor Invoice / CreateVendorCreditMemo
		[0.00, 0.00, 0.00, 0.95, 0.00, 0.00, 0.00, 0.00, 0.00, 0.05], # 5 Change Payment Term
		[0.00, 1.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.15], # 6 Change Vendor
		[0.00, 0.00, 0.00, 0.95, 0.00, 0.00, 0.00, 0.00, 0.00, 0.05], # 7 Change Quantity
		[0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 1.00], # 8 PostVendorAccountCreditItem / PostVendorAccountDebitItem /  Clear VendorAccountItem / 
		[0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 1.00]  # 9 Terminate
	])



	step = 0 
	params = get_params()
	latest_date = datetime(params['pr_req_year'], params['pr_req_month'], random.randint(1, 28)).date()
	#print(f'this is the start before create contract {latest_date}')
	latest_time = helpers.generate_random_time()
	purchasing = purchasing_doc_data.Purchasing(params=params, start_date=latest_date, index=i)
	preq_creation_date = latest_date

	#CREATE CONTRACT
	purchasing.create_contract(
		aedat=latest_date,
		ernam=get_user_name(params['autoprobs'][2])
	)
	latest_date += helpers.UPTO_WEEK()
	#print(f'date after create contract {latest_date}')
	pr_automation_rate = 0 if params['is_free_text'] else 3
	
	#CREATE PR
	if params['has_pr'] == True:
		purchasing.create_purchase_requisition_item(
			badat=latest_date, 
			ernam=get_user_name(params['autoprobs'][pr_automation_rate])
		)
	
	latest_date += helpers.UPTO_4DAYS()
	#print(f'date after create pr {latest_date}')
	creation_time = helpers.generate_random_time()
	
	#CREATE PO 
	if params['has_po'] == True:
		purchasing.create_purchase_order(
			aedat=latest_date,
			ernam=get_user_name(params['autoprobs'][2]),
			utime=creation_time
		)
	else:
		step = 3 #straight to goods receipt without PO
	
	creation_time = helpers.add_time(creation_time, helpers.UPFROM_4_TO_8_HOURS())
	purchase_creation_date = latest_date
	latest_date += helpers.UPTO_4DAYS()
	#print(f'date after create PO {latest_date}')
	total = sum([params['quantities'][j]*params['prices'][j] for j in range(len(params['matnrs']))])
	#print(total)
	if total > 320 and step == 0: #Hotel Chocolat value
	# if total > 38000: #Meritor value
		transition_prob[step][1] = 4.0



	while step != 9:
		if step == 0:
			#print(f'date before step 0 {latest_date}')
			if (params['is_free_text']) or (not params['has_contract']):
				if random.random() < 0.3:
					transition_prob[step][6] = 2.0
					latest_date += helpers.UPTO_4DAYS() 
					latest_date += helpers.UPTO_WEEK() 
				else:
					latest_date += helpers.UPTO_WEEK() 
			#print(f'date after step 0 {latest_date}')
		elif step == 1:
			#print(f'date before step approve PO {latest_date}')
			purchasing.approve_purchase_order(
				aedat=latest_date,
				ernam = get_user_name(params['autoprobs'][1])
			)
			latest_date += helpers.UPTO_4DAYS()
			#print(f'date after step 1 approve PO {latest_date}')
		elif step == 2:
			#print(f'date before step 2 send PO {latest_date}')
			purchasing.send_purchase_order(
				usnam=get_user_name(params['autoprobs'][1]),
				erdat=latest_date
			)
			latest_date += helpers.UPTO_WEEK()
			#print(f'date after step 2 send PO {latest_date}')
			if params['has_payment_term_change']:
				transition_prob[step][5] = 1.0

			if params['has_qty_change']: # 50% chance of Change Quantity
				transition_prob[step][7] = 1.0

		elif step == 3:
			#print(f'date before step 3 goods receipt {latest_date}')
			purchasing.goods_receipt(
				cpudt=latest_date,
				usnam=get_user_name(params['autoprobs'][6]),
				atime=latest_time,
				udate=purchase_creation_date,
				utime=creation_time
			)
			latest_date += helpers.UPTO_4DAYS()
			#print(f'date after step 3 goods receipt {latest_date}')

		elif step == 4:
			purchasing.create_vendor_invoice(
				cpudt=latest_date,
				ernam=get_user_name(0.9),
			)
			latest_date += helpers.UPTO_4DAYS()
			if params['has_creditmemo'] == True:
				purchasing.create_vendor_creditmemo(
					cpudt=latest_date,
					ernam=get_user_name(0.9),
			)
			
			latest_date += helpers.UPTO_MONTH()
		elif step == 5:
			purchasing.change_payment_term( # likely goes next to 'Post GR'
				udate=latest_date,
				ernam=get_user_name(0.2)
			)
			latest_date += helpers.UPTO_MONTH()
			
			if params['lifnr'] in ('VND30','VND29','VND28'):
				transition_prob[step][8] = 1.5

		elif step == 6:
			#print(f'date before step 6 change vendor {latest_date}')	
			purchasing.change_vendor( # likely goes to 'Approve PO'
				udate=latest_date,
				ernam=get_user_name(params['autoprobs'][1])
			)
			latest_date += helpers.UPTO_4DAYS()
			#print(f'date after step 6 change vendor {latest_date}')	
		elif step == 7:
			old_quantities = params['quantities']
			new_quantity_lines = random.sample(range(len(old_quantities)), random.randint(0, len(old_quantities)))
			new_quanity_quantities = [round(old_quantities[j]*(1+random.random())) for j in new_quantity_lines]
			purchasing.change_quantity( # likely goes to 'Send PO'
				badat=latest_date,
				ernam=get_user_name(params['autoprobs'][0]),
				line_numbers=new_quantity_lines,
				line_quantities=new_quanity_quantities
			)
			latest_date += helpers.UPTO_4DAYS()
		
		elif step == 8:
			purchasing.PostVendorAccountCreditItem(
				cpudt=latest_date,
				usnam=get_user_name(0.9),
			)
			if params['has_creditmemo'] == True:
				purchasing.PostVendorAccountDebitItem(  # TEST 
					cpudt=latest_date,
					usnam=get_user_name(0.9),
				)



		# normalize transition matrix
		transition_prob = transition_prob / transition_prob.sum(axis=1, keepdims=True)
		step = np.random.choice(10, p=transition_prob[step])

	for k, v in purchasing.tables.items():
		for entry_key in list(v.keys()):
			purchasing_doc_tables[k][entry_key] = purchasing.tables[k][entry_key]

KeyError: 'MAT322'

In [ ]:
for table, rows in purchasing_doc_tables.items():
    table_name = table.split('_')[0]
    all_cols = pd.DataFrame(columns=[c[0] for c in ctl.fetch_table(table_name)])
    all_cols_req = ctl.clean_columns(all_cols,table_name)
    df = pd.concat([all_cols_req, pd.DataFrame(rows.values())])
    df.to_csv(f'data/P2P/OCPM/purchasing-document/{table_name}.csv', index=False)